# Preparando as lições para o treinamento por aprendizado de máquina

Nesse notebook, a partir de parametros estabelecidos, as lições serão preparadas para então ser realizado o treinamento, para então serem realizadas as validações e inferências do modelo para detecção de faixas de pedestre.

In [ ]:
tamanho_janela = 32 # tamanho em pixels
sobreposicao_de_janelas = 16 # sobreposição das janelas em pixesl